In [3]:
import pandas as pd
import numpy as np

#save and load as parquet (for big files)
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
# I drop rows with 0 value of trading for this v2
trade_mx = pd.read_parquet(r"..\data\FAOSTAT\clean_data\food_trading\trade_matrix__imports_clean_codesOnly_v2.parquet")

In [5]:
distance_df = pd.read_csv(r"..\data\FAOSTAT\clean_data\food_trading\country_matrix_v4_final.csv")

In [6]:
distance_df.head()

,country_from,continent_from,country_to,Continent_to,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method,country_from_code,country_to_code,internal_distance
0,afghanistan,asia,afghanistan,asia,515.733111,1,1.0,0,land,2,2,515.733111
1,afghanistan,asia,albania,europe,4335.926901,0,0.0,0,water,2,3,NaN
2,afghanistan,asia,algeria,africa,5855.362822,0,0.0,0,water,2,4,NaN
3,afghanistan,asia,angola,africa,7596.963990,0,0.0,0,water,2,7,NaN
4,afghanistan,asia,argentina,south america,15273.883835,0,0.0,0,water,2,9,NaN


In [7]:
trade_mx.head()

,Reporter Country Code,Partner Country Code,Item Code,Year,Value_tons,Flag
0,4,2,231,2005,3.00,A
1,3,3,56,2005,52.00,A
2,7,2,176,2022,25.06,A
3,4,2,711,1996,171.00,A
4,4,2,711,1997,160.00,A


# Convert variables type

To decrease memory usage

In [8]:
#Convert continent to category
cols = ['continent_from', 'Continent_to']
for col in cols:
    distance_df[col] = distance_df[col].astype('category')

# Clean trade matrix
## to make it smaller

Create keys to merge

In [9]:
distance_df['key'] = distance_df['country_from_code'].astype(str) + '-' + distance_df['country_to_code'].astype(str)
trade_mx['key'] = trade_mx['Reporter Country Code'].astype(str) + '-' + trade_mx['Partner Country Code'].astype(str)

## I have to drop countries that don`t exist in trade matrix cause results where not ok. I had mixed continents.

I drop NaN in "share_border" and "-1" in contry codes from distance_df

In [10]:
distance_df.dropna(subset=['share_border'], inplace=True)

In [11]:
distance_df = distance_df[(distance_df['country_from_code'] > 0) & (distance_df['country_to_code'] > 0)]

In [12]:
full_trade_mx = trade_mx.merge(distance_df, on='key', how='inner')

In [13]:
full_trade_mx.head()

,Reporter Country Code,Partner Country Code,Item Code,Year,Value_tons,Flag,key,country_from,continent_from,country_to,Continent_to,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method,country_from_code,country_to_code,internal_distance
0,4,2,231,2005,3.00,A,4-2,algeria,africa,afghanistan,asia,5855.362822,0,0.0,0,water,4,2,NaN
1,3,3,56,2005,52.00,A,3-3,albania,europe,albania,europe,108.698297,1,1.0,0,land,3,3,108.698297
2,7,2,176,2022,25.06,A,7-2,angola,africa,afghanistan,asia,7596.963990,0,0.0,0,water,7,2,NaN
3,4,2,711,1996,171.00,A,4-2,algeria,africa,afghanistan,asia,5855.362822,0,0.0,0,water,4,2,NaN
4,4,2,711,1997,160.00,A,4-2,algeria,africa,afghanistan,asia,5855.362822,0,0.0,0,water,4,2,NaN


In [14]:
cols = ['country_from_code', 'country_to_code', 'internal_distance', 'key', 'country_from', 'country_to', 'continent_from', 'Continent_to']
full_trade_mx.drop(columns=cols, inplace=True)

In [15]:
full_trade_mx.to_parquet(r"..\data\FAOSTAT\clean_data\food_trading\trade_matrix__imports_clean_distance_v3.parquet", index=False)